In [1]:
import pandas as pd
from proxy_requests import ProxyRequests
import requests
from bs4 import BeautifulSoup as soup
import time

In [2]:
test_url = "https://fbref.com/en/matches/bf52349b/Fulham-Arsenal-September-12-2020-Premier-League"
r = ProxyRequests(test_url)
print(r.get())
page = soup(r.get_raw(), "html.parser")
page

None



<!DOCTYPE html>

<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '__tcfapiLocator';
	    var queue = [];
	    var win = window;
	    var cmpFrame;
	    
	    function addFrame() {
		var doc = win.document;
		var otherCMP = !!(win.frames[TCF_LOCATOR_NAME]);
		
		if (!o

In [56]:
def get_all_match_data(page):
    match_data = get_general_match_data(page)
    
    relevant_data = page.findAll("div",{"class":"section_wrapper"})
    home_data,home_id = get_team_data(relevant_data[0])
    away_data,away_id = get_team_data(relevant_data[1])
    
    # Each is a list of dictionaries, each dict is a row of df
    list_of_series = []
    for d in home_data:
        d["id"] = home_id
        series = pd.Series(d)
        list_of_series.append(series)
    for d in away_data:
        d["id"] = away_id
        series = pd.Series(d)
        list_of_series.append(series)
    df = pd.DataFrame(list_of_series)
    
    match_data_duplicated = {}
    for key in match_data.keys():
        item = match_data[key]
        match_data_duplicated[key] = [item]*len(df)
        
    df_match_data = pd.DataFrame(match_data_duplicated)
    
    df_final = pd.concat([df,df_match_data],axis=1)
    return df_final

def get_general_match_data(page):
    relevant_data = page.findAll("div",{"class":"scorebox"})[0]
    divs = relevant_data.findAll("div",recursive=False)
    home_data = divs[0]
    away_data = divs[1]
    generic_data = divs[2]
    d = {}
    
    d["home_name"] = str(home_data.find("a").text)
    d["home_score"] = int(home_data.find("div",{"class":"score"}).text)
    d["home_xg"] = float(home_data.find("div",{"class":"score_xg"}).text)
    d["home_manager"] = str(home_data.find("div",{"class":"datapoint"}).text.split(": ")[-1])

    d["away_name"] = str(away_data.find("a").text)
    d["away_score"] = int(away_data.find("div",{"class":"score"}).text)
    d["away_xg"] = float(away_data.find("div",{"class":"score_xg"}).text)
    d["away_manager"] = str(away_data.find("div",{"class":"datapoint"}).text.split(": ")[-1])
    
    d["date"] = str(generic_data.find("a").text)
    d["time"] = str(generic_data.find("span").text.split(" (")[0])
    d["matchweek"] = int(str(generic_data.findAll("div")[1].text.split(" (")[1].split(" ")[-1])[:-1])
    d["stadium"] = str(generic_data.findAll("small")[1].text)
    spans = generic_data.findAll("span",{"style":"display:inline-block"})
    d["ref"] = str(spans[0].text.split(" (")[0])
    d["var_ref"] = str(spans[-1].text.split(" (")[0])
    
    return d

def get_team_data(team_wrapper):
    team_id = team_wrapper["id"].split("_")[-1]
    
    summary_html = team_wrapper.find("div",{"id":"all_stats_" + team_id + "_summary"}).tbody
    summary = read_table_data(summary_html)
    
    passing_html = team_wrapper.find("div",{"id":"all_stats_" + team_id + "_passing"}).tbody
    passing = read_table_data(passing_html)
    
    passing_types_html = team_wrapper.find("div",{"id":"all_stats_" + team_id + "_passing_types"}).tbody
    passing_types = read_table_data(passing_types_html)
    
    defense_html = team_wrapper.find("div",{"id":"all_stats_" + team_id + "_defense"}).tbody
    defense = read_table_data(defense_html)
    
    possession_html = team_wrapper.find("div",{"id":"all_stats_" + team_id + "_possession"}).tbody
    possession = read_table_data(possession_html)
    
    misc_html = team_wrapper.find("div",{"id":"all_stats_" + team_id + "_misc"}).tbody
    misc = read_table_data(misc_html)
    
    output = combine_team_data([summary,passing,passing_types,defense,possession,misc])
    return output,team_id
    
def read_table_data(table_html):
    all_player_data = []
    player_html_list = table_html.findAll("tr")
    for player_html in player_html_list:
        player_data = read_player_table_data(player_html)
        all_player_data.append(player_data)
    # all_player_data is a list of dicts, 1 dict per player of their summary data
    return all_player_data

def read_player_table_data(player_html):
    a = {}
    a["name"] = player_html.th.a.text
    inner_data = player_html.findAll("td")
    a["pos"] = inner_data[2].text
    a["age"] = inner_data[3].text.split("-")[0]
    for i in range(4,len(inner_data)):
        a[str(inner_data[i]["data-stat"])] = inner_data[i].text
    return a

def combine_team_data(datasets):
    data_by_player = {}
    # First create dict key for each player name
    for d in datasets[0]:
        data_by_player[d["name"]] = []
    # Go through each dataset and update data_by_player
    for dataset in datasets:
        # Go through each player data and add to data_by_player dict of lists
        for i in range(len(dataset)):
            player_dict = dataset[i]
            player_name = player_dict["name"]
            data_by_player[player_name] += [player_dict]
    
    # Now each entry in data_by_player is a list of dictionaries of data points
    output_list = []
    for key in data_by_player.keys():
        p = data_by_player[key]
        merged_data = {**p[0],**p[1],**p[2],**p[3],**p[4],**p[5]}
        output_list += [merged_data]
        
    return output_list

In [57]:
t1 = time.time()
df = get_all_match_data(page)
t2 = time.time()
print("Took {} seconds".format(round(t2-t1,2)))
df.head()

Took 0.17 seconds


,name,pos,age,minutes,goals,assists,pens_made,pens_att,shots_total,shots_on_target,...,away_name,away_score,away_xg,away_manager,date,time,matchweek,stadium,ref,var_ref
0,Aboubakar Kamara,FW,25,62,0,0,0,0,1,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
1,Aleksandar Mitrović,FW,25,28,0,0,0,0,1,1,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
2,Ivan Cavaleiro,LW,26,90,0,0,0,0,0,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
3,Neeskens Kebano,RW,28,62,0,0,0,0,1,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote
4,Andre-Frank Zambo Anguissa,RW,24,28,0,0,0,0,0,0,...,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",12:30,1,"Craven Cottage, London",Chris Kavanagh,David Coote


In [58]:
df.shape

(28, 128)

In [59]:
print(list(df.columns))

['name', 'pos', 'age', 'minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'shots_total', 'shots_on_target', 'cards_yellow', 'cards_red', 'touches', 'pressures', 'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 'gca', 'passes_completed', 'passes', 'passes_pct', 'passes_progressive_distance', 'carries', 'carry_progressive_distance', 'dribbles_completed', 'dribbles', 'passes_total_distance', 'passes_completed_short', 'passes_short', 'passes_pct_short', 'passes_completed_medium', 'passes_medium', 'passes_pct_medium', 'passes_completed_long', 'passes_long', 'passes_pct_long', 'assisted_shots', 'passes_into_final_third', 'passes_into_penalty_area', 'crosses_into_penalty_area', 'progressive_passes', 'passes_live', 'passes_dead', 'passes_free_kicks', 'through_balls', 'passes_pressure', 'passes_switches', 'crosses', 'corner_kicks', 'corner_kicks_in', 'corner_kicks_out', 'corner_kicks_straight', 'passes_ground', 'passes_low', 'passes_high', 'passes_left_foot', 'passes_right_

In [61]:
cols = list(df.columns)
to_front = ['id', 'home_name', 'home_score', 'home_xg', 'home_manager', 'away_name', 'away_score', 
            'away_xg', 'away_manager', 'date', 'time', 'matchweek', 'stadium', 'ref', 'var_ref']
rest = [x for x in cols if x not in to_front]
print(to_front)
print(rest)
df_finalised = df[to_front + rest]
print(list(df_finalised.columns))
df_finalised.head()

['id', 'home_name', 'home_score', 'home_xg', 'home_manager', 'away_name', 'away_score', 'away_xg', 'away_manager', 'date', 'time', 'matchweek', 'stadium', 'ref', 'var_ref']
['name', 'pos', 'age', 'minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'shots_total', 'shots_on_target', 'cards_yellow', 'cards_red', 'touches', 'pressures', 'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 'gca', 'passes_completed', 'passes', 'passes_pct', 'passes_progressive_distance', 'carries', 'carry_progressive_distance', 'dribbles_completed', 'dribbles', 'passes_total_distance', 'passes_completed_short', 'passes_short', 'passes_pct_short', 'passes_completed_medium', 'passes_medium', 'passes_pct_medium', 'passes_completed_long', 'passes_long', 'passes_pct_long', 'assisted_shots', 'passes_into_final_third', 'passes_into_penalty_area', 'crosses_into_penalty_area', 'progressive_passes', 'passes_live', 'passes_dead', 'passes_free_kicks', 'through_balls', 'passes_pressure', 'passes_switches'

,id,home_name,home_score,home_xg,home_manager,away_name,away_score,away_xg,away_manager,date,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,fd962109,Fulham,0,0.2,Scott Parker,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",...,2,1,0,0,0,0,4,2,1,66.7
1,fd962109,Fulham,0,0.2,Scott Parker,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",...,1,1,0,0,0,0,0,0,0,
2,fd962109,Fulham,0,0.2,Scott Parker,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",...,1,4,0,0,0,0,6,1,0,100.0
3,fd962109,Fulham,0,0.2,Scott Parker,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",...,1,1,1,0,0,0,4,0,0,
4,fd962109,Fulham,0,0.2,Scott Parker,Arsenal,3,1.8,Mikel Arteta,"Saturday September 12, 2020",...,0,0,0,0,0,0,2,0,1,0.0
